In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display

# --- Activation and Cost Function ---
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(predictions, targets):
    return np.mean((predictions - targets) ** 2)

# --- Layer Class ---
class Layer:
    def __init__(self, number, size):
        self._number = number
        self._size = size
        self._data = None 

    @property
    def number(self):
        return self._number

    @property
    def size(self):
        return self._size

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_data):
        self._data = new_data

# --- Weight Class ---
class Weight:
    def __init__(self, from_layer, to_layer):
        self._value = np.random.randn(from_layer.size, to_layer.size).astype(np.float32)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        self._value = new_value

# --- Bias Class ---
class Bias:
    def __init__(self, to_layer):
        self._value = np.random.randn(1, to_layer.size).astype(np.float32)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        self._value = new_value

# --- Neural Network Class ---
class Neural:
    def __init__(self, activation):
        self._activation = activation
        self._layers = []
        self._weights = []
        self._biases = []

    def add_layer(self, layer):
        self._layers.append(layer)
        if len(self._layers) > 1:
            self._weights.append(Weight(self._layers[-2], self._layers[-1]))
            self._biases.append(Bias(self._layers[-1]))

    def forward(self):
        for i in range(1, len(self._layers)):
            prev_data = self._layers[i-1].data
            w = self._weights[i-1].value
            b = self._biases[i-1].value
            z = np.dot(prev_data, w) + b
            activated = self._activation(z)
            self._layers[i].data = activated

    @property
    def output(self):
        return self._layers[-1].data

# --- Load Data and Preprocess Batch ---
df = pd.read_csv('mnist_test.csv')
labels = df['label'].values
pixels = df.drop('label', axis=1).values.astype(np.float32) / 255.0

# --- Take a batch of images (first 5 for example) ---
batch_size = 5
x_batch = pixels[:batch_size]
y_labels = labels[:batch_size]

# --- Convert labels to one-hot vectors ---
y_batch = np.zeros((batch_size, 10))
for i, label in enumerate(y_labels):
    y_batch[i, label] = 1

# --- Show first image as preview ---
display(Image.fromarray((x_batch[0].reshape(28, 28) * 255).astype(np.uint8)))
print("Labels:", y_labels)

# --- Build Network ---
net = Neural(sigmoid)

net.add_layer(Layer(1, 784))   # Input
net.add_layer(Layer(2, 107))   # Hidden
net.add_layer(Layer(3, 26))    # Hidden
net.add_layer(Layer(4, 10))    # Output

net._layers[0].data = x_batch
net.forward()

# --- Output & Cost ---
output = net.output
print("\nPredictions:\n", output)
print("\nActual (One-Hot):\n", y_batch)
cost = mean_squared_error(output, y_batch)
print("\nMean Squared Error (Cost):", cost)

# --- Print Predicted Digits ---
predicted_digits = np.argmax(output, axis=1)
print("\nPredicted Digits:", predicted_digits)
